In [1]:
import os
import gc
import time

import pyreadr as py
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import torch
import torch.nn.functional as F

from torch import nn
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:5


> ### Data downloading
Data link  
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6C3JR1

##### Data description
Here we consoder dataset of "Additional Tennessee Eastman Process Simulation Data for Anomaly Detection Evaluation"
This dataverse contains the data referenced in Rieth et al. (2017). Issues and Advances in Anomaly Detection Evaluation for Joint Human-Automated Systems. To be presented at Applied Human Factors and Ergonomics 2017.
##### Columns description
* **faultNumber** ranges from 1 to 20 in the “Faulty” datasets and represents the fault type in the TEP. The “FaultFree” datasets only contain fault 0 (i.e. normal operating conditions).
* **simulationRun** ranges from 1 to 500 and represents a different random number generator state from which a full TEP dataset was generated (Note: the actual seeds used to generate training and testing datasets were non-overlapping).
* **sample** ranges either from 1 to 500 (“Training” datasets) or 1 to 960 (“Testing” datasets). The TEP variables (columns 4 to 55) were sampled every 3 minutes for a total duration of 25 hours and 48 hours respectively. Note that the faults were introduced 1 and 8 hours into the Faulty Training and Faulty Testing datasets, respectively.
* **columns 4-55** contain the process variables; the column names retain the original variable names.

In [2]:
# ! unzip ../../data/raw/dataverse_files.zip -d ../../data/raw/dataverse_files

In [3]:
#reading train data in .R format
a1 = py.read_r("../../data/raw/dataverse_files/TEP_FaultFree_Training.RData")
a2 = py.read_r("../../data/raw/dataverse_files/TEP_Faulty_Training.RData")

In [4]:
#reading test data in .R format
# a3 = py.read_r("../../data/raw/dataverse_files/TEP_FaultFree_Testing.RData")
# a4 = py.read_r("../../data/raw/dataverse_files/TEP_Faulty_Testing.RData")

In [5]:
print("Objects that are present in a1 :", a1.keys())
print("Objects that are present in a2 :", a2.keys())
# print("Objects that are present in a3 :", a3.keys())
# print("Objects that are present in a4 :", a4.keys())

Objects that are present in a1 : odict_keys(['fault_free_training'])
Objects that are present in a2 : odict_keys(['faulty_training'])


In [6]:
# concatinating the train and the test dataset

raw_train = pd.concat([a1['fault_free_training'], a2['faulty_training']])
# raw_test = pd.concat([a3['fault_free_testing'], a4['faulty_testing']])

In [7]:
# 5.250.000, 10.080.000
# len(raw_train), len(raw_test)

> ### Train-test-split

In [8]:
features = [
        'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
        'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 
        'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 
        'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
        'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 
        'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 
        'xmv_10', 'xmv_11'
    ]

In [9]:
raw_train['index'] = raw_train['faultNumber'] * 500 + raw_train['simulationRun'] - 1
# raw_test['index'] = raw_test['faultNumber'] * 500 + raw_test['simulationRun'] - 1

In [10]:
simulation_idx = raw_train[['index', 'faultNumber']].drop_duplicates()

In [11]:
X_train_idx, X_val_idx = train_test_split(simulation_idx['index'], 
                                          stratify=simulation_idx['faultNumber'],
                                          test_size=0.2, 
                                          random_state=42)

In [12]:
X_train = raw_train[raw_train['index'].isin(X_train_idx)].drop('index', axis=1)
X_val = raw_train[raw_train['index'].isin(X_val_idx)].drop('index', axis=1)
# X_test = raw_test.drop('index', axis=1)

> ### Scaling

In [13]:
scaler = StandardScaler()
scaler.fit(X_train[features])

X_train[features] = scaler.transform(X_train[features])
X_val[features] = scaler.transform(X_val[features])
# X_test[features] = scaler.transform(X_test[features])

> ### Dataset and dataloader

In [20]:
def correct(y_pred, target):
    
    y_pred = torch.softmax(y_pred, dim=1)
    y_pred = torch.max(y_pred, dim=1)[1]  
    
    return torch.eq(y_pred, target).sum().item()

In [21]:
# s_list = [0, 100, 200, 300, 400] #len = 5
# l_list = [1, 5, 10, 25, 50, 100] #len = 6


# for idx in range(239, 271): 

#     fault_sim_idx = idx // (len(s_list) * len(l_list))
    
#     start_length_idxs = idx % (len(s_list) * len(l_list))
    
#     print(fault_sim_idx, start_length_idxs)
#     start_idx = s_list[start_length_idxs // len(l_list)]
#     seq_length = l_list[start_length_idxs % len(l_list)]
#     print("start:", start_idx)
#     print("length:", seq_length)
    
#     X_train_tmp = X_train[X_train['index'] == fault_sim_idx].values[start_idx:start_idx+seq_length, :]
    
#     print(X_train_tmp.shape)

In [22]:
class DataTEP(Dataset):

    def __init__(self, X):
    
        self.X = X
        self.X = self.X.sort_values(['faultNumber', 'simulationRun', 'sample'])
        self.X['index'] = self.X.groupby(['faultNumber', 'simulationRun']).ngroup()
        self.X = self.X.set_index('index')
        
        self.s_list = [20, 100, 200, 300, 400]
#         self.s_list = [450]
#         self.l_list = [1, 5, 10, 25, 50, 100]
#         self.l_list = [10, 50]
        self.l_list = [100]
        
        self.features = [
                'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 
                'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 
                'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 
                'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 
                'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 
                'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'
            ]

    def __len__(self):
        return self.X.index.nunique() * len(self.s_list) * len(self.l_list)
    
    def __getitem__(self, idx):
        
        fault_sim_idx = idx // (len(self.s_list) * len(self.l_list))
    
        start_length_idxs = idx % (len(self.s_list) * len(self.l_list))
        
        start_idx = self.s_list[start_length_idxs // len(self.l_list)]
        seq_length = self.l_list[start_length_idxs % len(self.l_list)]
        
#         print(start_idx, start_idx+seq_length)

        features = self.X.loc[fault_sim_idx][self.features].values[start_idx : (start_idx+seq_length), :]
        target = self.X.loc[fault_sim_idx]['faultNumber'].unique()[0]

        features = torch.tensor(features, dtype=torch.float)
        target = torch.tensor(target, dtype=torch.long)

        return features, target

In [23]:
BATCH_SIZE = 64
NUM_CLASSES = 21

In [24]:
def collate_fn(batch):

    sequences = [x[0] for x in batch]
    labels = [x[1] for x in batch]
        
    lengths = torch.LongTensor([len(x) for x in sequences])
    lengths, idx = lengths.sort(0, descending=True)
    
    sequences = [sequences[i] for i in idx]
    
    labels = torch.tensor(labels, dtype=torch.long)[idx]
    
    sequences_padded = pad_sequence(sequences, batch_first=True)

    return sequences_padded, lengths, labels

In [25]:
train_ds = DataTEP(X_train)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

val_ds = DataTEP(X_val)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE*4, collate_fn=collate_fn)

# test_ds = DataTEP(X_test)
# test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE*4, collate_fn=collate_fn)

In [26]:
len(train_ds), len(val_ds)

(42000, 10500)

In [27]:
gc.collect()

14

> ### Model

In [28]:
class TwinModel(torch.nn.Module) :
    def __init__(self, NUM_LAYERS, INPUT_SIZE, HIDDEN_SIZE, LINEAR_SIZE, OUTPUT_SIZE, BIDIRECTIONAL, DEVICE):
        super().__init__()
        
        self.hidden_size = HIDDEN_SIZE
        self.num_layers = NUM_LAYERS
        self.input_size = INPUT_SIZE
        self.linear_size = LINEAR_SIZE
        self.output_size = OUTPUT_SIZE
        self.bidirectional = BIDIRECTIONAL
        
        self.lstm_1 = nn.LSTM(
                        input_size=self.input_size[0], 
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers, 
                        bidirectional=self.bidirectional,
                        batch_first=True,
                        dropout=0.4
            )
        
        self.lstm_2 = nn.LSTM(
                        input_size=self.input_size[1],
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers, 
                        bidirectional=self.bidirectional,
                        batch_first=True,
                        dropout=0.4
            )
        
        
        self.head = nn.Sequential(
                        nn.Linear(in_features=2*self.hidden_size*(self.bidirectional+1), out_features=self.linear_size),
                        nn.ReLU(),
                        nn.Dropout(p=0.4),
                        nn.Linear(in_features=self.linear_size, out_features=OUTPUT_SIZE),
            )
        
    def forward(self, x):
        
        x_1 = x[:, :, :41]
        x_2 = x[:, :, 41:]
        
        x_1, _ = self.lstm_1(x_1)
        x_2, __ = self.lstm_2(x_2)
        
        x_3 = torch.cat((x_1[:, -1], x_2[:, -1]), dim=-1)
        
        x = self.head(x_3)
        
        return x

In [29]:
class UniModel(torch.nn.Module) :
    def __init__(self, NUM_LAYERS, INPUT_SIZE, HIDDEN_SIZE, LINEAR_SIZE, OUTPUT_SIZE, BIDIRECTIONAL, DEVICE):
        super().__init__()
        
        self.hidden_size = HIDDEN_SIZE
        self.num_layers = NUM_LAYERS
        self.input_size = INPUT_SIZE
        self.linear_size = LINEAR_SIZE
        self.output_size = OUTPUT_SIZE
        self.bidirectional = BIDIRECTIONAL
        
        self.lstm = nn.LSTM(
                        input_size=self.input_size, 
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers, 
                        bidirectional=self.bidirectional,
                        batch_first=True,
                        dropout=0.4
            )
        
        self.head = nn.Sequential(
                        nn.Linear(in_features=self.hidden_size*(self.bidirectional+1), out_features=self.linear_size),
                        nn.ReLU(),
                        nn.Dropout(p=0.4),
                        nn.Linear(in_features=self.linear_size, out_features=OUTPUT_SIZE),
            )
        
    def forward(self, x, x_length):
        
        x_packed = pack_padded_sequence(x, x_length, batch_first=True)
        
        x_lstm_out, _ = self.lstm(x_packed)
        
        x_unpacked, _ = pad_packed_sequence(x_lstm_out, batch_first=True)
        
        x = self.head(x_unpacked[:, -1])
        
        return x

In [30]:
NUM_EPOCHS = 100
LEARNING_RATE = 0.001

NUM_LAYERS = 2
HIDDEN_SIZE = 256
LINEAR_SIZE = 128
BIDIRECTIONAL = True

In [31]:
model = UniModel(NUM_LAYERS=NUM_LAYERS, INPUT_SIZE=52, HIDDEN_SIZE=HIDDEN_SIZE, 
                 LINEAR_SIZE=LINEAR_SIZE, OUTPUT_SIZE=NUM_CLASSES, BIDIRECTIONAL=BIDIRECTIONAL, DEVICE=device)

# model = TwinModel(NUM_LAYERS=NUM_LAYERS, INPUT_SIZE=[41, 11], HIDDEN_SIZE=HIDDEN_SIZE,
#                  LINEAR_SIZE=LINEAR_SIZE, OUTPUT_SIZE=NUM_CLASSES, BIDIRECTIONAL=BIDIRECTIONAL, DEVICE=device)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=25, gamma=0.5)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', verbose=True)

torch.manual_seed(42)

In [32]:
for i, (X_batch, X_lengths, y_batch) in enumerate(train_dl):
    if i < 1:
        print(type(X_batch), type(X_lengths), type(y_batch))
        print(len(X_batch), len(X_lengths), len(y_batch))
        X_batch, y_batch_train = X_batch.to(device), y_batch.to(device)
        y_pred_train = model(X_batch, X_lengths)
        print("y_batch_train.size()", y_batch.size())
        print("y_pred_train.size()", y_pred_train.size(), '\n')
    else:
        break

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
64 64 64
y_batch_train.size() torch.Size([64])
y_pred_train.size() torch.Size([64, 21]) 



In [33]:
# !rm -r runs

> ### Training

In [34]:
# writer = SummaryWriter(comment=f'NL{NUM_LAYERS}_H{HIDDEN_SIZE}_L{LINEAR_SIZE}_B{BIDIRECTIONAL}_S{-100}')

writer = SummaryWriter(comment=f'window_100_cut_start')

In [35]:
loss_train_all, loss_val_all = [], []
accuracy_train_all, accuracy_val_all = [], []

for epoch in range(NUM_EPOCHS):

    start = time.time()
    print(f'Epoch: {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}\n')
#     print(f'Epoch: {epoch}\n')

    loss_train_epoch, loss_val_epoch = 0, 0
    correct_train_epoch, correct_val_epoch = 0, 0
    n_train, n_val = 0, 0

    model.train()
    for (X_batch_train, X_batch_lengths_train, y_batch_train) in tqdm(train_dl):

        X_batch_train, X_batch_lengths_train, y_batch_train =\
                    X_batch_train.to(device), X_batch_lengths_train.to(device), y_batch_train.to(device)

        optimizer.zero_grad()
        y_pred_train = model(X_batch_train, X_batch_lengths_train)
        loss_train = criterion(y_pred_train, y_batch_train)
        loss_train.backward()
        optimizer.step()

        loss_train_epoch += loss_train.item() * y_batch_train.size()[0]
        correct_train_epoch += correct(y_pred_train, y_batch_train)
        n_train += y_batch_train.size()[0]

    scheduler.step()
    model.eval()

    with torch.no_grad():
        
        for item in model.parameters():
            print(item.grad.mean())
        
        for (X_batch_val, X_batch_lengths_val, y_batch_val) in tqdm(val_dl):

            X_batch_val, X_batch_lengths_val, y_batch_val =\
                    X_batch_val.to(device), X_batch_lengths_val.to(device), y_batch_val.to(device)

            y_pred_val = model(X_batch_val, X_batch_lengths_val)
            loss_val = criterion(y_pred_val, y_batch_val)
            
            loss_val_epoch += loss_val.item() * y_batch_val.size()[0]
            correct_val_epoch += correct(y_pred_val, y_batch_val)
            n_val += y_batch_val.size()[0]
            

    loss_mean_train_epoch = loss_train_epoch / n_train
    loss_mean_val_epoch = loss_val_epoch / n_val

    loss_train_all.append(loss_mean_train_epoch)
    loss_val_all.append(loss_mean_val_epoch)

    accuracy_train_epoch = correct_train_epoch / n_train
    accuracy_val_epoch = correct_val_epoch / n_val

    accuracy_train_all.append(accuracy_train_epoch)
    accuracy_val_all.append(accuracy_val_epoch)

    writer.add_scalars('LOSS per epoch', {"train": loss_mean_train_epoch, "val": loss_mean_val_epoch}, epoch)
    writer.add_scalars('ACCURACY per epoch', {"train": accuracy_train_epoch, "val": accuracy_val_epoch}, epoch)
    
#     scheduler.step(loss_mean_val_epoch)
    
    end = time.time()
    print(f"epoch time: {end - start}")  
    print(f"mean loss train: {loss_mean_train_epoch}, mean loss val: {loss_mean_val_epoch}")
    print(f"accuracy train: {accuracy_train_epoch}, accuracy val: {accuracy_val_epoch}")

    print("---------------------------------------------------------------------------------------------------")

Epoch: 0, Learning Rate: 0.001




tensor(-9.6609e-06, device='cuda:5')
tensor(2.7035e-06, device='cuda:5')
tensor(-0.0006, device='cuda:5')
tensor(-0.0006, device='cuda:5')
tensor(-8.7892e-07, device='cuda:5')
tensor(1.1910e-07, device='cuda:5')
tensor(-2.6242e-05, device='cuda:5')
tensor(-2.6242e-05, device='cuda:5')
tensor(7.5466e-07, device='cuda:5')
tensor(7.8357e-07, device='cuda:5')
tensor(-0.0001, device='cuda:5')
tensor(-0.0001, device='cuda:5')
tensor(9.3478e-09, device='cuda:5')
tensor(0., device='cuda:5')
tensor(-2.9383e-05, device='cuda:5')
tensor(-2.9383e-05, device='cuda:5')
tensor(-1.3563e-07, device='cuda:5')
tensor(0.0002, device='cuda:5')
tensor(1.0187e-09, device='cuda:5')
tensor(-1.4192e-09, device='cuda:5')



epoch time: 163.15401649475098
mean loss train: 1.3667882441566104, mean loss val: 0.6768577030499776
accuracy train: 0.5241190476190476, accuracy val: 0.7214285714285714
---------------------------------------------------------------------------------------------------
Epoch: 1, Learning Rate: 0.001




tensor(-1.1898e-05, device='cuda:5')
tensor(1.3487e-06, device='cuda:5')
tensor(0.0002, device='cuda:5')
tensor(0.0002, device='cuda:5')
tensor(9.3765e-06, device='cuda:5')
tensor(5.5119e-09, device='cuda:5')
tensor(-2.3836e-05, device='cuda:5')
tensor(-2.3836e-05, device='cuda:5')
tensor(-5.6271e-09, device='cuda:5')
tensor(2.0308e-06, device='cuda:5')
tensor(-8.6925e-05, device='cuda:5')
tensor(-8.6925e-05, device='cuda:5')
tensor(1.1806e-07, device='cuda:5')
tensor(0., device='cuda:5')
tensor(5.6772e-05, device='cuda:5')
tensor(5.6772e-05, device='cuda:5')
tensor(6.9305e-06, device='cuda:5')
tensor(-0.0004, device='cuda:5')
tensor(2.9372e-09, device='cuda:5')
tensor(1.5966e-09, device='cuda:5')



epoch time: 143.4803864955902
mean loss train: 0.5730537755829948, mean loss val: 0.4396893901370821
accuracy train: 0.7667380952380952, accuracy val: 0.8037142857142857
---------------------------------------------------------------------------------------------------
Epoch: 2, Learning Rate: 0.001



...


epoch time: 135.25319600105286
mean loss train: 0.0025520727686397593, mean loss val: 0.24335161855391094
accuracy train: 0.9991190476190476, accuracy val: 0.9656190476190476
---------------------------------------------------------------------------------------------------
Epoch: 98, Learning Rate: 0.000125




tensor(-1.1446e-08, device='cuda:5')
tensor(5.4080e-09, device='cuda:5')
tensor(1.4247e-06, device='cuda:5')
tensor(1.4247e-06, device='cuda:5')
tensor(8.1173e-09, device='cuda:5')
tensor(6.7357e-10, device='cuda:5')
tensor(1.7063e-08, device='cuda:5')
tensor(1.7063e-08, device='cuda:5')
tensor(6.2317e-10, device='cuda:5')
tensor(4.6700e-10, device='cuda:5')
tensor(6.6993e-08, device='cuda:5')
tensor(6.6993e-08, device='cuda:5')
tensor(-1.4270e-11, device='cuda:5')
tensor(0., device='cuda:5')
tensor(-5.2918e-09, device='cuda:5')
tensor(-5.2918e-09, device='cuda:5')
tensor(2.4917e-09, device='cuda:5')
tensor(-8.9970e-07, device='cuda:5')
tensor(9.2024e-10, device='cuda:5')
tensor(1.7068e-09, device='cuda:5')



epoch time: 134.48392534255981
mean loss train: 0.004163508067256771, mean loss val: 0.23043907694873356
accuracy train: 0.9986428571428572, accuracy val: 0.9653333333333334
---------------------------------------------------------------------------------------------------
Epoch: 99, Learning Rate: 0.000125




tensor(-3.2052e-06, device='cuda:5')
tensor(-2.1345e-07, device='cuda:5')
tensor(-0.0001, device='cuda:5')
tensor(-0.0001, device='cuda:5')
tensor(1.5717e-07, device='cuda:5')
tensor(4.1969e-08, device='cuda:5')
tensor(3.6290e-05, device='cuda:5')
tensor(3.6290e-05, device='cuda:5')
tensor(2.0317e-08, device='cuda:5')
tensor(-1.2657e-07, device='cuda:5')
tensor(-3.1529e-05, device='cuda:5')
tensor(-3.1529e-05, device='cuda:5')
tensor(-1.4370e-09, device='cuda:5')
tensor(0., device='cuda:5')
tensor(1.0326e-06, device='cuda:5')
tensor(1.0326e-06, device='cuda:5')
tensor(-3.2030e-07, device='cuda:5')
tensor(-3.5300e-05, device='cuda:5')
tensor(-1.3706e-10, device='cuda:5')
tensor(-2.7718e-10, device='cuda:5')



epoch time: 134.4112846851349
mean loss train: 0.0025597341079708364, mean loss val: 0.24006785838199513
accuracy train: 0.9991666666666666, accuracy val: 0.9648571428571429
---------------------------------------------------------------------------------------------------


In [36]:
model.eval()

y_ans_val, y_true_val = [], []

with torch.no_grad():

    for (X_batch_val, y_batch_val) in tqdm(val_dl):

        X_batch_val, y_batch_val = X_batch_val.to(device), y_batch_val.to(device)

        y_pred_val = model(X_batch_val)
        
        y_pred_prob = F.softmax(y_pred_val.cpu(), dim=-1)
        y_pred_class = y_pred_prob.max(dim=-1)[1]
       
        y_ans_val += y_pred_class.tolist()
        y_true_val += y_batch_val.tolist()

ValueError: too many values to unpack (expected 2)

In [ ]:
plt.figure(figsize=(15, 10))
plt.title("FDR")
sns.heatmap(confusion_matrix(y_true_val, y_ans_val, normalize='pred'), annot=True, cmap=sns.cm.rocket_r)
plt.xlabel('predicted class')
plt.ylabel('true class')
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("loss")
plt.plot(np.arange(len(loss_train_all)), loss_train_all, '-o', marker='.', label='train')
plt.plot(np.arange(len(loss_val_all)), loss_val_all, '-o', marker='.', label='val')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("accuracy")
plt.plot(np.arange(len(accuracy_train_all)), accuracy_train_all, '-o', marker='.', label='train')
plt.plot(np.arange(len(accuracy_val_all)), accuracy_val_all, '-o', marker='.', label='val')
plt.legend()
plt.show()

# ----------------------------------------------------------------------------------------